In [3]:
!pip install pandas
!pip install vaderSentiment

zsh:1: /Users/amks/Documents/classes/182/182proj/env/bin/pip: bad interpreter: /Users/amks/Desktop/182proj/env/bin/python3: no such file or directory
zsh:1: /Users/amks/Documents/classes/182/182proj/env/bin/pip: bad interpreter: /Users/amks/Desktop/182proj/env/bin/python3: no such file or directory


In [4]:
import json
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

ModuleNotFoundError: No module named 'pandas'

In [57]:
# Opening and reading json file
dict_list = []
# json_file = './shakespeare_cleaned_combined.json'
json_file = './rj_combined.json'
# output_file = './groundtruth_outputs/shakespeare_cleaned_combined_vader_truth.csv'
output_file = './groundtruth_outputs/rj_combined_vader_truth.csv'
data = ''
with open(json_file, 'r') as f:
    data = json.load(f)

In [58]:
# creating data frame of all dialogues while using vader sentiment analysis
speaker = []
listener = []
dialogue = []
sentiment_score = []

analyzer = SentimentIntensityAnalyzer()
for exchange in data:
    speaker += [exchange['speaker']]
    listener += [exchange['listener']]
    text = exchange['dialogue']
    dialogue += [text]
    
    vs = analyzer.polarity_scores(text)
    sentiment_score += [vs['compound']]

df_dialogue = pd.DataFrame.from_dict({'speaker': speaker,
                                      'listener': listener,
                                      'dialogue': dialogue,
                                      'dialogue_sentiment': sentiment_score})
df_dialogue

,speaker,listener,dialogue,dialogue_sentiment
0,CHORUS,SAMPSON,"Two households, both alike in dignity, In fair...",-0.5719
1,SAMPSON,CHORUS,"Gregory, on my word, we’ll not carry coals.",0.0000
2,SAMPSON,GREGORY,"Gregory, on my word, we’ll not carry coals.",0.0000
3,GREGORY,SAMPSON,"No, for then we should be colliers.",0.0000
4,GREGORY,SAMPSON,"No, for then we should be colliers.",0.0000
...,...,...,...,...
1681,MONTAGUE,CAPULET,"But I can give thee more, For I will raise her...",0.6956
1682,MONTAGUE,CAPULET,"But I can give thee more, For I will raise her...",0.6956
1683,CAPULET,MONTAGUE,"As rich shall Romeo’s by his lady’s lie, Poor ...",0.1280
1684,CAPULET,PRINCE,"As rich shall Romeo’s by his lady’s lie, Poor ...",0.1280


In [59]:
# finding the mean sentiment between two characters
df_dialogue_mean = (df_dialogue.drop(columns=['dialogue'])
                               .groupby(['speaker', 'listener'])
                               .agg(np.mean)
                               .reset_index().rename(columns={'dialogue_sentiment' : 'avg_sentiment'}))
df_dialogue_mean 

,speaker,listener,avg_sentiment
0,ABRAM,GREGORY,0.000000
1,ABRAM,SAMPSON,-0.075822
2,APOTHECARY,ROMEO,-0.178857
3,APOTHECARY,"WELL, JULIET, I WILL LIE WITH THEE TONIGHT",0.000000
4,BALTHASAR,FRIAR LAWRENCE,0.004083
...,...,...,...
194,TYBALT,ROMEO,-0.079000
195,WE SHALL BE MUCH UNFURNISH’D FOR THIS TIME,CAPULET,0.000000
196,WE SHALL BE MUCH UNFURNISH’D FOR THIS TIME,NURSE,0.000000
197,"WELL, JULIET, I WILL LIE WITH THEE TONIGHT",APOTHECARY,-0.871500


In [60]:
# creating the vader ground truth sentiment
df_dialogue_vader_truth = pd.merge(left=df_dialogue,
                                   right=df_dialogue_mean,
                                   left_on=['speaker', 'listener'],
                                   right_on=['speaker', 'listener'],
                                   how='left')
df_dialogue_vader_truth

,speaker,listener,dialogue,dialogue_sentiment,avg_sentiment
0,CHORUS,SAMPSON,"Two households, both alike in dignity, In fair...",-0.5719,-0.571900
1,SAMPSON,CHORUS,"Gregory, on my word, we’ll not carry coals.",0.0000,0.000000
2,SAMPSON,GREGORY,"Gregory, on my word, we’ll not carry coals.",0.0000,-0.062038
3,GREGORY,SAMPSON,"No, for then we should be colliers.",0.0000,-0.087993
4,GREGORY,SAMPSON,"No, for then we should be colliers.",0.0000,-0.087993
...,...,...,...,...,...
1681,MONTAGUE,CAPULET,"But I can give thee more, For I will raise her...",0.6956,0.264300
1682,MONTAGUE,CAPULET,"But I can give thee more, For I will raise her...",0.6956,0.264300
1683,CAPULET,MONTAGUE,"As rich shall Romeo’s by his lady’s lie, Poor ...",0.1280,-0.127267
1684,CAPULET,PRINCE,"As rich shall Romeo’s by his lady’s lie, Poor ...",0.1280,0.062967


In [61]:
df_dialogue_vader_truth.to_csv(output_file)